### Code Explanation

#### Cell 12 - **Model Completion Operations**
This cell defines the `ModelCompletionOperation` class, which is responsible for performing text and image-based model completion tasks using OpenAI APIs.

**Key Features**:
1. **Initialization (`__init__`)**:
   - Reads the OpenAI API key from the project configuration and initializes the OpenAI client.
   - Sets default parameters like `max_tokens` and `temperature` for API calls.

2. **Methods**:
   - `GetModel()`: Returns the initialized OpenAI client.
   - `RunVision(initModel, targetPrompt, targetImage)`:
     - Converts the input image to a base64-encoded string for API compatibility.
     - Sends a multi-modal prompt (image and text) to the OpenAI GPT-4 model.
     - Handles the response and extracts generated text, or returns an error if the API call fails.

#### Cell 13 - **Recognizer Registry and Main Processing**
This cell implements classes to define custom recognizers and register them for text analysis and anonymization using Presidio.

**Key Classes**:
1. **`GetRecognizers`**:
   - Instantiates and configures recognizers for:
     - Numbers (`NUMBER`)
     - Credit cards (`CREDIT_CARD`)
     - Emails (`EMAIL_ADDRESS`)
     - URLs (`URL`)
     - Phone numbers (`PHONE_NUMBER`)
   - Supports multiple languages, including English (`en`), Spanish (`es`), French (`fr`), and more.

2. **`GetRegistry`**:
   - Manages a `RecognizerRegistry` instance to register all custom recognizers, including:
     - Predefined recognizers such as `PERSONNAMERECOGNIZER`.
     - Custom recognizers like `NUMBERRECOGNIZER`, `CREDITCARDRECOGNIZER`, etc.

**Steps in Main Processing**:
- Add each custom recognizer to the registry, enabling comprehensive text analysis for identifying sensitive entities.

### Usage
- The `ModelCompletionOperation` class can handle text and image completion tasks using GPT-4.
- The `GetRecognizers` and `GetRegistry` classes integrate with Presidio to detect and anonymize sensitive entities from text, tailored to specific patterns (e.g., numbers, emails, URLs).


In [ ]:
# Update Cell 12 - Model Completion Operations
from openai import OpenAI

class ModelCompletionOperation(object):
    def __init__(self)->CLASSINIT:
        self.openaiapikey = project_config["apikey"]["OPENAI_API_KEY"]
        self.client = OpenAI(api_key=self.openaiapikey)
        self.defaultMaxToken = 100
        self.temperature = 0.1

    def GetModel(self)->MODEL:
        return self.client

    def RunVision(self, initModel:MODEL, targetPrompt:str, targetImage:IMAGE)->RESULT:
        try:
            # Convert image to base64
            _, buffer = cv2.imencode(".jpg", targetImage)
            encoded_frame = base64.b64encode(buffer).decode('utf-8')

            # Create API call
            response = initModel.chat.completions.create(
                model="gpt-4o",
                messages=[{
                    "role": "user",
                    "content": [
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{encoded_frame}"
                            }
                        },
                        {
                            "type": "text",
                            "text": targetPrompt
                        }
                    ]
                }],
                max_tokens=self.defaultMaxToken,
                temperature=self.temperature
            )

            # Extract response text
            if response.choices:
                return response.choices[0].message.content
            else:
                return "No response generated"

        except Exception as e:
            print(f"OpenAI API Error: {str(e)}")
            return f"Error: {str(e)}"

In [ ]:
# Cell 13 - Main Process Selection
from presidio_analyzer.recognizer_registry import RecognizerRegistry
from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine
from presidio_analyzer.nlp_engine import NlpEngineProvider
from presidio_analyzer.context_aware_enhancers import LemmaContextAwareEnhancer

CONTEXTAWARE = LemmaContextAwareEnhancer(context_similarity_factor=0.2,
                                        min_score_with_context_similarity=0.2)

class GetRecognizers(object):
    def __init__(self)->CLASSINIT:
        self.supportedLanguage = ["en", "es", "fr", "de", "ru", "nl", "xx"]
        self.numberRecognizer = NUMBERRECOGNIZER(supported_entities=["NUMBER"],
                                               supported_language=self.supportedLanguage)
        self.creditcardRecognizer = CREDITCARDRECOGNIZER(supported_entities=["CREDIT_CARD"],
                                                       supported_language=self.supportedLanguage)
        self.emailRecognizer = EMAILRECOGNIZER(supported_entities=["EMAIL_ADDRESS"],
                                             supported_language=self.supportedLanguage)
        self.urlRecognizer = URLRECOGNIZER(supported_entities=["URL"],
                                         supported_language=self.supportedLanguage)
        self.phoneRecognizer = PHONENUMBERRECOGNIZER(supported_entities=["PHONE_NUMBER"],
                                                   supported_language=self.supportedLanguage)

class GetRegistry(object):
    def __init__(self)->CLASSINIT:
        self.recognizers = GetRecognizers()
        self.registry = RecognizerRegistry()

    def Run(self)->PROCESS:
        self.registry.add_recognizer(recognizer=self.recognizers.numberRecognizer)
        self.registry.add_recognizer(recognizer=self.recognizers.creditcardRecognizer)
        self.registry.add_recognizer(recognizer=self.recognizers.emailRecognizer)
        self.registry.add_recognizer(recognizer=self.recognizers.urlRecognizer)
        self.registry.add_recognizer(recognizer=self.recognizers.phoneRecognizer)
        self.registry.add_recognizer(recognizer=PERSONNAMERECOGNIZER)


In [ ]:
# Here's how to use the system:
'''
# Initialize the model operation
model_op = ModelCompletionOperation()
model = model_op.GetModel()

# Example with an image (you'll need to load an image first)
# image = cv.imread('your_image.jpg')
# result = model_op.RunVision(model, "What's in this image?", image)
# print(result)

# Initialize recognizers and registry
registry = GetRegistry()
registry.Run()
'''

'\n# Initialize the model operation\nmodel_op = ModelCompletionOperation()\nmodel = model_op.GetModel()\n\n# Example with an image (you\'ll need to load an image first)\n# image = cv.imread(\'your_image.jpg\')\n# result = model_op.RunVision(model, "What\'s in this image?", image)\n# print(result)\n\n# Initialize recognizers and registry\nregistry = GetRegistry()\nregistry.Run()\n'